In [1]:
"""
Purpose:
Author: Hasnain Cheena
Date: 1/1/2019
"""

'\nPurpose:\nAuthor: Hasnain Cheena\nDate: 1/1/2019\n'

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import tokenize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import pysrt

import nltk 

%matplotlib inline

VADER works better on the sentence level than when classifying whole texts. Need to check accuracy of 30s increments. 

VADER:
1. NEG - Proportion of negative text in the document
   NEU - Proportion of neutral text in the document
   POS - Proportion of positive text in the document 
   COMPOUND - -1(ex negative) to +1(ex positive) and 1.3/2.9 are supercool sentiment

In [3]:
#purpose of function: 
def return_dialog(sub):
    """
    Returns the dialog of the .srt file sub.
    Input :
        sub : name of the .srt file as a string, ex. 's01e01.srt'
    Output :
        dialog : dialog contained in .srt file as a single string
    """
    lines = []
    with open(sub) as file:
        start = 2 
        for index,line in enumerate(file):
            if index >= start:
                # check if line contains dialog
                if (index == start) or ((index == start+1) and (len(line.split()) != 0)):
                    lines.append(line.strip('\n').replace('\'',''))
                    space_index = index
                if ((index > space_index + 2) and (len(line.split()) != 0)):
                    lines.append(line.strip('\n').replace('\'',''))
                if (len(line.split()) == 0):
                    space_index = index
    # output the dialog as a single string, removing the last two lines which contain attribution
    return ' '.join(lines[:-2])

In [4]:
hobbit2_input_path = r'C:\Users\hasna\Desktop\Smell of Fear\Subtitles_SRT\Walter.srt'
di = return_dialog(hobbit2_input_path)
hobbit_sub = pysrt.open(hobbit2_input_path,encoding='iso-8859-1')

In [12]:
#split the paragraph into a list of sentences use NLTK 
sentences = tokenize.sent_tokenize(di)

#import movie runtimes
movie_runtime_path = r'C:\Users\hasna\Desktop\Smell of Fear\NumericalData\movie_runtimes.csv'
movie_runtime_df = pd.read_csv(movie_runtime_path, usecols = ['movie', 'runtime', 'runtime (mins)', 'effective runtime'], header = 0, nrows = 6)
movies_30s_list = list() #entire list of movies with each movie in its 30s split config

#split movie subs into 30s increments  
hobbit_30s_incr = list() #list to store a particular movie in its 30s split config 

counter = 0
start_sec = 0
start_min = 0
end_min = 0
end_sec = 30

while(counter < movie_runtime_df.loc[3]['effective runtime']): 
    part = hobbit_sub.slice(starts_after = {'minutes': start_min, 'seconds': start_sec}, ends_before = {'minutes': end_min, 'seconds': end_sec})
    dialogue = str(part.text) #convert subrip to str making it easier to edit 
    modified_dialog = str()
    #remove \n 
#     for i in dialogue:
#         if i != '\n':
#             modified_dialog = modified_dialog + i
#         elif i == '\n':
#             modified_dialog = modified_dialog + ' '
        
    hobbit_30s_incr.append(dialogue)
     
    counter += 1

    start_sec += 30
    end_sec += 30
    if(start_sec >= 60):
        start_sec = 0
        start_min += 1
    if(end_sec >= 60):
        end_sec = 0
        end_min += 1

In [6]:
# VADER Sentiment Analyzer 
analyzer = SentimentIntensityAnalyzer()

In [13]:
hobbit_30s_incr

['',
 '',
 '',
 '',
 '',
 '',
 '',
 "How can I help you today?\nHi. I can't seem to leave\na wink for someone.\nI don't know, is my page broken?\nDo I... do I maybe have\na broken page or...?\nI've never heard\nof that, but okay.\nYou're trying to use\nyour eHarmony account\nfor the first time?\nI am.\nOkay. I'm looking\nat your profile.\nWe have a pretty intricate\nmatching algorithm.\nThat's what distinguishes us from\nother online dating services.\nYeah, I like that.\nActually, I'm just trying\nto leave a wink for one person.",
 'She started in my division\nat work about a month ago,\nand I overheard her\nnear the bagels\nsaying she was on your site.\nOkay, that\'s unique,\nbut let me ask you,\nyou left a lot of this stuff,\nlike... like the "Been There,\nDone That" section,\nyou left it blank.\nYeah, I think I skipped it.\nOkay, well, you got to\nhelp me out here, man.\nDon\'t skip stuff.\nOkay, well, I haven\'t\nreally been anywhere...\nnoteworthy or mentionable.\nHave you...\nhav

In [7]:
#analyze the entire script
pol = analyzer.polarity_scores(di)
print(pol)

{'neg': 0.065, 'neu': 0.713, 'pos': 0.222, 'compound': 1.0}


In [8]:
#analyze each sentence
for s in sentences:
    print(s)
    pol = analyzer.polarity_scores(s)['compound']
    print(pol)
    print()

Hi.
0.0

Todd Maher, eHarmony.
0.0

How can I help you today?
0.4019

Hi.
0.0

I cant seem to leave a wink for someone.
0.0382

I dont know, is my page broken?
-0.4767

Do I... do I maybe have a broken page or...?
-0.4767

Ive never heard of that, but okay.
0.3291

Youre trying to use your eHarmony account for the first time?
0.0

I am.
0.0

Okay.
0.2263

Im looking at your profile.
0.0

We have a pretty intricate matching algorithm.
0.5859

Thats what distinguishes us from other online dating services.
0.0

Yeah, I like that.
0.5719

Actually, Im just trying to leave a wink for one person.
-0.0516

Cheryl Melhoff?
0.0

She started in my division at work about a month ago, and I overheard her near the bagels saying she was on your site.
0.0

Okay, thats unique, but let me ask you, you left a lot of this stuff, like... like the "Been There, Done That" section, you left it blank.
0.5719

Yeah, I think I skipped it.
0.296

Okay, well, you got to help me out here, man.
0.6908

Dont skip st

No, I just, um...
-0.296

I used to...
0.0

I was kind of into the scene.
0.0

Like...
0.0

The skateboard scene?
0.0

Yeah.
0.296

Wow.
0.5859

Hi, Peg.
0.0

Yeah, Sean OConnell.
0.296

We actually, need to find him.
0.0

Yes.
0.4019

If you could hurry.
0.0

Kickflip.
0.0

Yeah.
0.296

Its a good one for the repertoire.
0.4404

Im Walter.
0.0

Rich.
0.5574

Yeah, we think he might know something or have an asset we need.
0.5719

You know, you just got to get a little more snap, get that thing up there.
0.0

Want me to, show you something?
0.0772

Sure.
0.3182

Here.
0.0

Like your trucks.
0.3612

Thanks.
0.4404

Check this out.
0.0

Know about the flick?
0.0

Yeah, we-we-weve finished processing his April work order.
0.296

Yes, its kind of important.
0.4951

Okay.
0.2263

Did you forward the job 1090s?
0.0

Or his last salary?
0.0

Wow!
0.6239

Okay.
0.2263

Wow.
0.5859

Here you go.
0.0

That was awesome.
0.6249

So it could be either place.
0.0

Thanks, Walter.
0.4404

Okay.
0.226


The photograph.
0.0

Now.
0.0

Let me see it.
0.0

He looked it up.
0.0

Theres no such thing as a "silver bath."
-0.296

Listen, there was uncertainty about the negative.
-0.7269

I thought...
0.0

I understood that, hopefully, it was in Iceland...
0.4019

It was in your care!
0.5411

If you just give me some more time... Youre fired.
-0.5945

Well make another cover.
0.2732

Hey!
0.0

More than a million... way more... negatives have come through my office.
0.0

I have never mislaid one.
0.0

Put it on a plaque.
0.0

Hang it at your next job.
0.0

Now I got to explain this to the board?
0.0

Yeah, Im sure Sean OConnell will be thrilled.
0.7506

Hey... Tell me again... who is this guy?
0.0

You okay?
0.2263

Yeah.
0.296

Thats cool.
0.3182

Its a skateboard.
0.0

Its for Cheryls son.
0.0

Cheryls not in.
0.0

She was let go.
0.0

I told you, man, its bad.
-0.5423

Hey!
0.0

Thats my fern.
0.0

Thank you.
0.3612

Hi.
0.0

Hi.
0.0

Can I...?
0.0

Yeah, is, um... Is this, um... Im looki

In [9]:
#analyse each 30s increments
for inc in hobbit_30s_incr:
    print(inc)
    pol = analyzer.polarity_scores(inc)['compound']
    print(pol)
    print()


0.0


0.0


0.0


0.0


0.0


0.0


0.0

How can I help you today? Hi. I can't seem to leave a wink for someone. I don't know, is my page broken? Do I... do I maybe have a broken page or...? I've never heard of that, but okay. You're trying to use your eHarmony account for the first time? I am. Okay. I'm looking at your profile. We have a pretty intricate matching algorithm. That's what distinguishes us from other online dating services. Yeah, I like that. Actually, I'm just trying to leave a wink for one person.
0.9428

She started in my division at work about a month ago, and I overheard her near the bagels saying she was on your site. Okay, that's unique, but let me ask you, you left a lot of this stuff, like... like the "Been There, Done That" section, you left it blank. Yeah, I think I skipped it. Okay, well, you got to help me out here, man. Don't skip stuff. Okay, well, I haven't really been anywhere... noteworthy or mentionable. Have you... have you done anything
0.9587

Hello

In [11]:
di

'Hi. Todd Maher, eHarmony. How can I help you today? Hi. I cant seem to leave a wink for someone. I dont know, is my page broken? Do I... do I maybe have a broken page or...? Ive never heard of that, but okay. Youre trying to use your eHarmony account for the first time? I am. Okay. Im looking at your profile. We have a pretty intricate matching algorithm. Thats what distinguishes us from other online dating services. Yeah, I like that. Actually, Im just trying to leave a wink for one person. Cheryl Melhoff? She started in my division at work about a month ago, and I overheard her near the bagels saying she was on your site. Okay, thats unique, but let me ask you, you left a lot of this stuff, like... like the "Been There, Done That" section, you left it blank. Yeah, I think I skipped it. Okay, well, you got to help me out here, man. Dont skip stuff. Okay, well, I havent really been anywhere... noteworthy or mentionable. Have you... have you done anything noteworthy, mentionable? Hello